In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import nashpy as nash
import pulp
import math as math
from scipy.optimize import linprog
from itertools import combinations

# Set-Up:

- Dataframe Loading
- Base Tradeoff Matrix
- Team - Vector Conversion Functions
- Example Teams

In [2]:
# Global Variables, don't change these, unless you are me:

payoff_df = pd.read_csv("MarvelRivals_Payoff_Matrix.csv")
hero_list = hero_list = list(payoff_df.iloc[:, 0])

# Base Tradeoff Matrix
payoff_df = pd.read_csv("MarvelRivals_Payoff_Matrix.csv",  index_col=0)
payoff_matrix = payoff_df.values

# Convert a list of character names (length 6) into a binary bector
def team_to_vector(selected_heroes):
    vector = np.zeros(len(hero_list), dtype=int)
    for i, hero in enumerate(hero_list):
        if hero in selected_heroes:
            vector[i] = 1
    return vector

# Convert a binary bector into a list of character names (length 6)
def vector_to_team(binary_vector):
    selected_heroes = [hero_list[i] for i in range(len(hero_list)) if binary_vector[i] == 1]
    return selected_heroes

# Example Teams
Avengers = ["Black Widow", "Iron Man", "Thor", "Hulk", "Hawkeye", "Captain America"]
FantasticFour = ["Invisible Woman", "The Thing", "Human Torch", "Mister Fantastic", "Spider Man", "Wolverine"]
Guardians = ["Mantis", "Rocket Raccoon", "Adam Warlock", "Star Lord", "Groot", "Venom"]
Xmen = ["Magneto", "Psylocke", "Wolverine", "Magik", "Storm", "Namor"]
RandomTeam = ["Magneto", "Doctor Strange", "The Punisher", "Scarlet Witch", "Cloak & Dagger", "Loki"]

bestTeam = ['Captain America', 'Iron Fist', 'Magik', 'Storm', 'Mantis', 'Rocket Raccoon']

In [3]:
# Example Usage
MyTeam = ['Doctor Strange', 'Groot', 'The Punisher', 'Winter Soldier', "Cloak & Dagger", "Invisible Woman"]
OpponentTeam = ["Doctor Strange", "Peni Parker", "Magik", "Moon Knight", "Cloak & Dagger", "Invisible Woman"]

rowPlayer = team_to_vector(Guardians)
columnPlayer = team_to_vector(Guardians)

result = rowPlayer.T @ payoff_matrix @ columnPlayer
result

np.float64(0.0)

# 1.0: Base Question: Optimal Counter Strategy given known opponent Strategy

In this section, we solve the most fundamental problem in Marvel Rivals team composition:

**Given a known opponent team, how do we select 6 heroes that maximize our expected performance against them?**

This is framed as a **binary optimization problem** using a precomputed **payoff matrix** `A`, where each entry `A[i, j]` measures how well hero *i* performs against hero *j*.

---

#### Problem Setup

- Let $ x ∈ {(0,1)}^{37} $ be our binary decision vector for team selection.
- Let $ y ∈ {(0,1)}^{37} $ be the opponent's team vector (with exactly 6 ones).
- Let $ A ∈ ℝ^{37×37} $ be the payoff matrix.

#### We aim to:

- Maximize: xᵀ A y
- Subject to: sum(x) == 6
- sum(y) == 6

It turns out, given an opponent team, it is relatively easy to figure out the optimal conter team using cvxpy.

In [4]:
# In order to use, first use team_to_vector() to convert the list of characters into a binary vector
def base_optimal_counter(opponent_team):
    x = cp.Variable(37, boolean=True)
    y = opponent_team
    constraint_0 = [cp.sum(x) == 6]
    constraint_1 = [cp.sum(y) == 6]
    
    obj = cp.Maximize(x.T @ payoff_matrix @ y)
    prob = cp.Problem(obj,constraint_0+constraint_1)
    prob.solve()

    return x.value

def base_optimal_counter_info(opponent_team):
    n = 37
    x = cp.Variable(n, boolean=True)
    y = opponent_team
    constraint_0 = [cp.sum(x) == 6]
    constraint_1 = [cp.sum(y) == 6]
    
    obj = cp.Maximize(x.T @ payoff_matrix @ y)
    prob = cp.Problem(obj,constraint_0+constraint_1)
    prob.solve()

    optimal_team = vector_to_team(x.value.round())
    expected_value = x.value.T @ payoff_matrix @ y

    print("Optimal Team:")
    for hero in optimal_team:
        print(f"-{hero}")
    print(f"Expected Utility Score of (x^T * A * y) is: {expected_value:.2f}")
    return x.value

# This finds an optimal conter team whose members are excluded from banned_list (see session 1.2)
def optimal_counter_with_ban(opponent_team, banned_list):

    n = 37
    if np.any((opponent_team + banned_list) > 1):
        raise ValueError("Invalid input: A hero cannot be both in opponent_team and banned_list!")
    
    x = cp.Variable(n, boolean=True)
    constraint_0 = [cp.sum(x) == 6]
    constraint_1 = [banned_list + x <= 1] 

    obj = cp.Maximize(cp.sum(cp.multiply(x, payoff_matrix @ opponent_team)))

    prob = cp.Problem(obj, constraint_0 + constraint_1)
    prob.solve()

    counter_team_vector = x.value.round()
    counter_team_heroes = vector_to_team(counter_team_vector)

    expected_value = np.sum(np.multiply(counter_team_vector, payoff_matrix @ opponent_team))

    print("Optimal Counter Team (Without Banned Heroes):")
    for hero in counter_team_heroes:
        print(f"- {hero}") 

    print(f"Expected Utility Score with Ban (x^T * A * y): {expected_value:.2f}")

    return counter_team_vector

In [5]:
# Example Usage:
OpponentTeam = ["Doctor Strange", "Peni Parker", "Magik", "Moon Knight", "Cloak & Dagger", "Invisible Woman"]
y = team_to_vector(bestTeam)
print("Best Conter to OpponentTeam:", ', '.join(bestTeam), "is:")
OpponentTeamConter = base_optimal_counter_info(y)

Best Conter to OpponentTeam: Captain America, Iron Fist, Magik, Storm, Mantis, Rocket Raccoon is:
Optimal Team:
-Captain America
-Peni Parker
-Magik
-Storm
-Mantis
-Rocket Raccoon
Expected Utility Score of (x^T * A * y) is: 0.12


In [6]:
optimal_ban = team_to_vector(['Loki', 'Lunar Snow', "Wolverine", "Peni Parker"])
y = team_to_vector(bestTeam) 
result_with_ban = optimal_counter_with_ban(y, optimal_ban)

Optimal Counter Team (Without Banned Heroes):
- Captain America
- Iron Fist
- Magik
- Storm
- Mantis
- Rocket Raccoon
Expected Utility Score with Ban (x^T * A * y): 0.00


# 1.1: Secondary Question: Optimal Solution to Von Neumann Min Max Equation

In zero-sum games, the **Von Neumann Minimax Theorem** guarantees the existence of an optimal strategy that a player can adopt to **maximize their worst-case outcome**, regardless of what the opponent chooses.

---

#### The Minimax Formulation

Let `A` be the **payoff matrix**, where each entry `A[i, j]` represents how well hero *i* performs against hero *j*. Then the minimax objective is:

$$
\max_{\mathbf{x}} \ \min_{\mathbf{y}} \ \mathbf{x}^T A \mathbf{y}
$$

- `x` is our strategy (a 6-hero team, encoded as a binary vector of length 37)
- `y` is the opponent’s strategy (also a 6-hero team)
- The game is **zero-sum**, meaning if we gain, the opponent loses equally.

This equation models the idea:  
> What is the best team we can pick **assuming the opponent will counter us in the worst way possible**?

---

#### Why Minimax?

Unlike the base counter strategy (which assumes the opponent team is known), this problem assumes **we don’t know the opponent's team**, and instead optimize for the **safest possible team**, one that performs reliably across the board.

This is especially useful for a team of new players who has no idea about the game.

---

#### Brute-Force Strategy

The number of possible teams is \( \binom{37}{6} = 2324784 \), which is computationally feasible for one side if we iterate smartly.

Our function `min_max_solution(payoff_matrix)` does the following:

1. Loops through every 6-hero combination as our potential team.
2. For each of those, it estimates the **worst-case payoff** the opponent can respond with.
3. Keeps track of the team that guarantees the highest worst-case performance.

In [23]:
# 37 choose 6 is not very big so use Brutal Force Method:
def min_max_solution_brutal(payoff_matrix):
    n = payoff_matrix.shape[0]
    assert payoff_matrix.shape == (37, 37), "Expected a 37x37 payoff matrix, if it is bigger, this will take much longer."
    
    best_value = float('-inf')
    best_vector = None
    
    for row_subset in combinations(range(n), 6):
        c = np.sum(payoff_matrix[list(row_subset), :], axis=0)
        
        chosen_cols = np.partition(c, 6)[:6]
        payoff = np.sum(chosen_cols)
        
        if payoff > best_value:
            best_value = payoff
            vector = np.zeros(n, dtype=int)
            vector[list(row_subset)] = 1
            best_vector = vector

    print(f"Row-player guaranteed payoff is greater than: {best_value:.2f}")
    BestTeam = vector_to_team(vector)
    print("Von Neumann MinMax Best team according to brutal force strategy is:", ', '.join(BestTeam))
    
    return best_vector

In [24]:
# This uses greedy descent so it should save a lot more time and scales better:
def min_max_solution_greedy(payoff_matrix):
    n = payoff_matrix.shape[0]

    current_team = np.zeros(n, dtype=int)
    current_team[:6] = 1

    improvement = True
    while improvement:
        opponent = base_optimal_counter(current_team)
        opponent = np.round(opponent)

        current_score = current_team.T @ payoff_matrix @ opponent

        improvement = False
        best_swap_score = current_score
        best_team = current_team.copy()

        current_indices = np.where(current_team == 1)[0]
        remaining_indices = np.where(current_team == 0)[0]

        for out_hero in current_indices:
            for in_hero in remaining_indices:
                temp_team = current_team.copy()
                temp_team[out_hero] = 0
                temp_team[in_hero] = 1

                temp_score = temp_team.T @ payoff_matrix @ base_optimal_counter(temp_team)

                if temp_score > best_swap_score:
                    best_swap_score = temp_score
                    best_team = temp_team.copy()
                    improvement = True

        current_team = best_team

    BestTeam = vector_to_team(current_team)
    print("Von Neumann MinMax Best team according to greedy descent strategy is:", ', '.join(BestTeam))
    return current_team


In [25]:
OptimalVector = min_max_solution_brutal(payoff_matrix)
print("Best Conter to von Neumann optimal team is:")
BestTeamConter = base_optimal_counter_info(OptimalVector)

Row-player guaranteed payoff is greater than: -0.12
Von Neumann MinMax Best team according to brutal force strategy is: Captain America, Iron Fist, Magik, Storm, Mantis, Rocket Raccoon
Best Conter to von Neumann optimal team is:
Optimal Team:
-Captain America
-Peni Parker
-Magik
-Storm
-Mantis
-Rocket Raccoon
Expected Utility Score of (x^T * A * y) is: 0.12


In [26]:
OptimalVector = min_max_solution_greedy(payoff_matrix)
print("Best Conter to von Neumann optimal team is:")
BestTeamConter = base_optimal_counter_info(OptimalVector)

Von Neumann MinMax Best team according to greedy descent strategy is: Captain America, Iron Fist, Magik, Storm, Mantis, Rocket Raccoon
Best Conter to von Neumann optimal team is:
Optimal Team:
-Captain America
-Peni Parker
-Magik
-Storm
-Mantis
-Rocket Raccoon
Expected Utility Score of (x^T * A * y) is: 0.12


# 1.2: Secondary Question: Optimal Character Ban Strategy

In the game, once you hit a certain rank (by being good at the game), the game introduce a new mechanism where a team can collectively ban two characters where the opponent team can no longer choose, this makes the character counter strategy more interesting.

#### Objective

Given a known team composition `your_team`, we want to:

> **Determine which two heroes should be banned to minimize the opponent’s ability to counter us.**

This is formulated as a binary optimization problem where:
- `b` is a binary vector indicating **which 2 heroes to ban**
- `x` is a binary vector representing the **opponent’s best response team**, given the bans

We solve:
- Minimize: xᵀ A y Subject to: sum(b) == 2
- sum(x) == 6
- b + y ≤ 1 (cannot ban someone already on our team)

In [27]:
def who_to_ban(your_team):
    n = 37
    b = cp.Variable(n, boolean=True) 
    x = cp.Variable(n, boolean=True) 

    constraint_0 = [cp.sum(b) == 2]  
    constraint_1 = [cp.sum(x) == 6]  
    constraint_2 = [b + your_team <= 1]

    obj = cp.Minimize(cp.sum(cp.multiply(x, payoff_matrix @ your_team)))

    prob = cp.Problem(obj, constraint_0 + constraint_1 + constraint_2)
    prob.solve()

    banned_vector = b.value.round()
    banned_heroes = np.where(banned_vector == 1)[0]  # Indices of banned heroes
    banned_hero_name = vector_to_team(banned_vector)

    opponent_team = x.value.round()

    print("Heroes to Ban:")
    for hero in banned_hero_name:
        print(f"- {hero}") 

    return banned_vector

In [30]:
banned_list = who_to_ban(OptimalVector)
result_with_ban = optimal_counter_with_ban(OptimalVector, banned_list)

Heroes to Ban:
- Loki
- Luna Snow
Optimal Counter Team (Without Banned Heroes):
- Captain America
- Peni Parker
- Magik
- Storm
- Mantis
- Rocket Raccoon
Expected Utility Score with Ban (x^T * A * y): 0.12


# 2.0: Base Question: Optimal Strategies for Team Ups Constraints

Problem: Maximize the number of possible team ups, with constraints:
- A team consists of 6 characters
- maximize win rate while as many heroes as possible is in a team up


team ups are powerful, but their strength is hard to quantify, therefore not very useful for solving the main problem: In this subproblem, we explore the ideal team composition where as many heroes as possible must be included in a team up while maintaining a standard team structure (2-3 strategists, 1-2 vanguards, 1-2 duelists) to maximize the team's win rate.

using the win rate of mathups

we were able to find 23 different teams with 5 team ups, which is the maximum number of team ups a team can get.

Using the matchup winrate matrix, we get this result: 
...


a team should consist of 2 of each class, but when this is enforced, we see that there is no team with 2 of each class with 5 team ups.


In [ ]:
team_ups = [
    ("Adam Warlock", "Star Lord"),
    ("Adam Warlock", "Mantis"),
    ("Thor", "Captain America"),
    ("Thor", "Storm"),
    ("Hela", "Loki"),
    ("Hela", "Thor"),
    ("Venom", "Spider Man"),
    ("Venom", "Peni Parker"),
    ("Hulk", "Doctor Strange"),
    ("Hulk", "Iron Man"),
    ("Rocket Raccoon", "The Punisher"),
    ("Rocket Raccoon", "Winter Soldier"),
    ("Invisible Woman", "The Thing"),
    ("Invisible Woman", "Mister Fantastic"),
    ("Magik", "Black Panther"),
    ("Magik", "Psylocke"),
    ("Human Torch", "Storm"),
    ("Iron Fist", "Luna Snow"),
    ("Spider Man", "Squirrel Girl"),
    ("Scarlet Witch", "Magneto"),
    ("Luna Snow", "Namor"),
    ("Luna Snow", "Jeff The Land Shark"),
    ("Groot", "Rocket Raccoon"),
    ("Groot", "Jeff The Land Shark"),
    ("Hulk", "Wolverine"),
    ("Invisible Woman", "Human Torch"),
    ("The Thing", "Wolverine"),
    ("Cloak & Dagger", "Moon Knight"),
    ("Hawkeye", "Black Widow")
]

win_rate_matrix = pd.read_csv("MarvelRivals_WinRate_matrix.csv", index_col=0)

heroes = sorted(hero_list)


vanguards = ["Captain America", "Doctor Strange", "Groot", "Hulk", "Magneto", "Peni Parker", "The Thing", "Thor", "Venom"]
duelists = ["Black Panther", "Black Widow", "Hawkeye", "Hela", "Human Torch", "Iron Fist", "Iron Man", "Magik", "Mister Fantastic",
            "Moon Knight", "Namor", "Psylocke", "Scarlet Witch", "Spider Man", "Squirrel Girl", "Star Lord", "Storm", "The Punisher", 
            "Winter Soldier","Wolverine"]
strategists = ["Adam Warlock", "Cloak & Dagger", "Invisible Woman", "Jeff The Land Shark", "Loki", "Luna Snow", "Mantis", "Rocket Raccoon"]

if len(vanguards) + len(duelists) + len(strategists) != len(heroes):
    raise ValueError("The number of heroes is not correct")

hero_ids = {hero: idx for idx, hero in enumerate(heroes)}

vanguard_ids = [hero_ids[hero] for hero in vanguards]
duelist_ids = [hero_ids[hero] for hero in duelists]
strategist_ids = [hero_ids[hero] for hero in strategists]

team_up_tuples = [(hero_ids[a], hero_ids[b]) for a, b in sorted(team_ups) if hero_ids[a] < hero_ids[b]]

hero_avg_win_rates = win_rate_matrix.mean(axis=1).values

def printsolution(problem, x, y):
    selected_heroes = [heroes[i] for i in range(len(heroes)) if x.value[i] > 0.5]

    selected_team_ups = [(heroes[i], heroes[j]) for (i, j) in team_up_tuples if y[(i, j)].value > 0.5]

    # Calculate the average win rate using hero names
    selected_hero_avg_win_rates = [
    hero_avg_win_rates[i] for i in range(len(heroes)) if x.value[i] > 0.5
]

# Print the individual averages
    print("\nSelected Heroes' Average Win Rates:")
    for hero, rate in zip(selected_heroes, selected_hero_avg_win_rates):
        print(f"- {hero}: {rate:.2f}%")

    average_win_rate = problem.objective.value

    print("\nSelected Heroes:")
    for hero in selected_heroes:
        print("-", hero, ": Vanguard" if hero in vanguards else "Duelist" if hero in duelists else "Strategist")

    print("\nSelected Team-Ups:")
    for duo in selected_team_ups:
        print("-", duo)

    print(f"\nAverage Win Rate of Selected Team-Ups: {average_win_rate/600:.2%}")
    


In [ ]:
def solve_teamups(teamUpCount, classLimit = False):
    x = cp.Variable(len(heroes), boolean=True) 
    y = {t: cp.Variable(boolean=True) for t in team_up_tuples}

    objective = cp.Maximize(cp.sum(cp.multiply(x, hero_avg_win_rates)))
    
    constraints = [
        cp.sum(x) == 6,
        cp.sum([y[t] for t in team_up_tuples]) == teamUpCount
    ]
    if classLimit:
        constraints.extend([
        cp.sum(x[vanguard_ids]) >= 1,   
        cp.sum(x[vanguard_ids]) <= 2,
        
        cp.sum(x[duelist_ids]) >= 1,
        cp.sum(x[duelist_ids]) <= 2,
        
        cp.sum(x[strategist_ids]) <= 3   
        ])
    
    # boolean AND operation
    for (i, j) in team_up_tuples:
        constraints.append(y[(i, j)] <= x[i])
        constraints.append(y[(i, j)] <= x[j])
        constraints.append(y[(i, j)] >= x[i] + x[j] - 1)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    if problem.status == cp.OPTIMAL:
        printsolution(problem, x, y)

In [ ]:
solve_teamups(5, classLimit=False)

optimal

Selected Heroes' Average Win Rates:
- Hulk: 52.22%
- Invisible Woman: 47.21%
- Iron Man: 48.98%
- Mister Fantastic: 49.50%
- The Thing: 45.46%
- Wolverine: 54.37%

Selected Heroes:
- Hulk : Vanguard
- Invisible Woman Strategist
- Iron Man Duelist
- Mister Fantastic Duelist
- The Thing : Vanguard
- Wolverine Duelist

Selected Team-Ups:
- ('Hulk', 'Iron Man')
- ('Hulk', 'Wolverine')
- ('Invisible Woman', 'Mister Fantastic')
- ('Invisible Woman', 'The Thing')
- ('The Thing', 'Wolverine')

Average Win Rate of Selected Team-Ups: 49.62%


In [ ]:
solve_teamups(5, classLimit=True)

infeasible


In [ ]:
solve_teamups(5, classLimit=False)

optimal

Selected Heroes' Average Win Rates:
- Hulk: 52.22%
- Invisible Woman: 47.21%
- Iron Man: 48.98%
- Mister Fantastic: 49.50%
- The Thing: 45.46%
- Wolverine: 54.37%

Selected Heroes:
- Hulk : Vanguard
- Invisible Woman Strategist
- Iron Man Duelist
- Mister Fantastic Duelist
- The Thing : Vanguard
- Wolverine Duelist

Selected Team-Ups:
- ('Hulk', 'Iron Man')
- ('Hulk', 'Wolverine')
- ('Invisible Woman', 'Mister Fantastic')
- ('Invisible Woman', 'The Thing')
- ('The Thing', 'Wolverine')

Average Win Rate of Selected Team-Ups: 49.62%


In [ ]:
solve_teamups(4, classLimit=True)

optimal

Selected Heroes' Average Win Rates:
- Groot: 50.94%
- Hulk: 52.22%
- Jeff The Land Shark: 51.85%
- Rocket Raccoon: 55.50%
- Winter Soldier: 53.12%
- Wolverine: 54.37%

Selected Heroes:
- Groot : Vanguard
- Hulk : Vanguard
- Jeff The Land Shark Strategist
- Rocket Raccoon Strategist
- Winter Soldier Duelist
- Wolverine Duelist

Selected Team-Ups:
- ('Groot', 'Jeff The Land Shark')
- ('Groot', 'Rocket Raccoon')
- ('Hulk', 'Wolverine')
- ('Rocket Raccoon', 'Winter Soldier')

Average Win Rate of Selected Team-Ups: 53.00%


# 3.0: Base Question: Ultimate Time Constraint / Win Rate optimization


In [ ]:
Ultimate_data = pd.read_csv('MarvelRivals_Ultimate_Matrix.csv')

ults_per_game = Ultimate_data['ults per game'].to_numpy()
winrate = Ultimate_data['Win Rate'].to_numpy()
character_name = Ultimate_data['Name'].to_list()
character_type = Ultimate_data['Type'].to_list()
types = ['Vanguards','Duelists','Strategists']
c_ik_list=[]
for i_idx,i in enumerate(character_name):
        c_ik_list.append((character_name[i_idx],character_type[i_idx]))
c_ik_list
average_ults_per_game = np.mean(ults_per_game)
average_winrate = np.mean(winrate)
average_ults_per_game,average_winrate
A=average_winrate/average_ults_per_game

In [ ]:
def one_to_one_best_team_comp():
    A=average_winrate/average_ults_per_game
    B=1/6
    num_types = len(types)
    num_chars = len(character_name)

    A_c_ik=np.ones(num_chars)
    b_ik=[6] #team comp constraint

    A_ck=np.zeros((num_types,num_chars))

    for k_idx,k in enumerate(types):
        for c_idx,c in enumerate(character_name):
            if c_ik_list[c_idx][1] == types[k_idx]:
                A_ck[k_idx][c_idx]=1
    
    b_ck=np.full(num_types,2,dtype=int)

    A_eq = np.vstack((A_c_ik,A_ck))
    b_eq = np.hstack((b_ik,b_ck))
    c=np.ones(num_chars)

    for i_idx,i in enumerate(character_name):
        c[i_idx]=-A*ults_per_game[i_idx]-B*winrate[i_idx]# -min = max
    
    results=linprog(c,A_eq=A_eq,b_eq=b_eq,bounds=(0,1),method='highs')

    team_comp = []
    u_i = []
    x_i= []
    
    for i_idx,i in enumerate(results.x):
        if i>0:
            team_comp.append(c_ik_list[i_idx])
            u_i.append(ults_per_game[i_idx])
            x_i.append(winrate[i_idx])

    total_ults = math.floor(np.sum(u_i))
    avg_winrate = np.mean(x_i)
    
    print('The best team composition is:',team_comp)
    print('The total ults per game is:',total_ults) 
    print('The average winrate is:',avg_winrate)

def highest_winrate_best_team_comp():
    A=1.5
    B=1/6
    num_types = len(types)
    num_chars = len(character_name)

    A_c_ik=np.ones(num_chars)
    b_ik=[6] #team comp constraint

    A_ck=np.zeros((num_types,num_chars))

    for k_idx,k in enumerate(types):
        for c_idx,c in enumerate(character_name):
            if c_ik_list[c_idx][1] == types[k_idx]:
                A_ck[k_idx][c_idx]=1
    
    b_ck=np.full(num_types,2,dtype=int)

    A_eq = np.vstack((A_c_ik,A_ck))
    b_eq = np.hstack((b_ik,b_ck))
    c=np.ones(num_chars)

    for i_idx,i in enumerate(character_name):
        c[i_idx]=-A*ults_per_game[i_idx]-B*winrate[i_idx]# -min = max
    
    results=linprog(c,A_eq=A_eq,b_eq=b_eq,bounds=(0,1),method='highs')

    team_comp = []
    u_i = []
    x_i= []
    
    for i_idx,i in enumerate(results.x):
        if i>0:
            team_comp.append(c_ik_list[i_idx])
            u_i.append(ults_per_game[i_idx])
            x_i.append(winrate[i_idx])

    total_ults = math.floor(np.sum(u_i))
    avg_winrate = np.mean(x_i)
    
    print('The best team composition is:',team_comp)
    print('The total ults per game is:',total_ults) 
    print('The average winrate is:',avg_winrate,'%')

def best_team_comp():
    A=1
    B=0
    num_types = len(types)
    num_chars = len(character_name)

    A_c_ik=np.ones(num_chars)
    b_ik=[6] #team comp constraint

    A_ck=np.zeros((num_types,num_chars))

    for k_idx,k in enumerate(types):
        for c_idx,c in enumerate(character_name):
            if c_ik_list[c_idx][1] == types[k_idx]:
                A_ck[k_idx][c_idx]=1
    
    b_ck=np.full(num_types,2,dtype=int)

    A_eq = np.vstack((A_c_ik,A_ck))
    b_eq = np.hstack((b_ik,b_ck))
    c=np.ones(num_chars)

    for i_idx,i in enumerate(character_name):
        c[i_idx]=-A*ults_per_game[i_idx]-B*winrate[i_idx]# -min = max
    
    results=linprog(c,A_eq=A_eq,b_eq=b_eq,bounds=(0,1),method='highs')

    team_comp = []
    u_i = []
    x_i= []
    
    for i_idx,i in enumerate(results.x):
        if i>0:
            team_comp.append(c_ik_list[i_idx])
            u_i.append(ults_per_game[i_idx])
            x_i.append(winrate[i_idx])

    total_ults = math.floor(np.sum(u_i))
    avg_winrate = np.mean(x_i)
    
    print('The best team composition is:',team_comp)
    print('The total ults per game is:',total_ults) 
    print('The average winrate is:',avg_winrate)

In [ ]:
one_to_one_best_team_comp()
highest_winrate_best_team_comp()
best_team_comp()

The best team composition is: [('Groot', 'Vanguards'), ('Loki', 'Strategists'), ('Storm', 'Duelists'), ('Venom', 'Vanguards'), ('Cloak & Dagger', 'Strategists'), ('Squirrel Girl', 'Duelists')]
The total ults per game is: 46
The average winrate is: 49.035000000000004
The best team composition is: [('Groot', 'Vanguards'), ('Loki', 'Strategists'), ('Storm', 'Duelists'), ('Venom', 'Vanguards'), ('Rocket Racoon', 'Strategists'), ('Squirrel Girl', 'Duelists')]
The total ults per game is: 46
The average winrate is: 50.30000000000001 %
The best team composition is: [('Groot', 'Vanguards'), ('Loki', 'Strategists'), ('Storm', 'Duelists'), ('Venom', 'Vanguards'), ('Cloak & Dagger', 'Strategists'), ('Squirrel Girl', 'Duelists')]
The total ults per game is: 46
The average winrate is: 49.035000000000004
